### Importing Libraries

In [1]:
import numpy as np
import os
import math
import lxml.html as lh
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import requests

#import sys
#!{sys.executable} -m pip install BeautifulSoup

### Web Scraping

In [2]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(URL)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [3]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [4]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d: %s'%(i,name))
    col.append((name,[]))

1: Postal code

2: Borough

3: Neighborhood



In [5]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content()
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [6]:
[len(C) for (title,C) in col]


[181, 181, 181]

In [7]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head()

,Postal code,Borough,Neighborhood
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront\n


In [8]:
df = df.replace(r'\n','', regex=True)
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


### Removing Not Assigned Boroughs

In [9]:
df = df.rename(columns = {'Postal code\n': 'PostalCode', 'Borough\n':'Borough', 'Neighborhood\n':'Neighborhood'})
df1= df[df['Borough'] != 'Not assigned']
df1.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


### Combining neighborhoods using a comma

In [10]:
df1 = df1.replace(r'/',',', regex=True)
df1.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


### Making the Neighborhood same as the Borugh for Not Assigned Neighborhood

In [11]:
for i in range(0,df1.shape[0]):
    if df1['Neighborhood'].iloc[i] == '':
        if df1['Borough'].iloc[i] != 'Not Assigned':
            df1['Neighborhood'].iloc[i] = df1['Borough'].iloc[i]

In [12]:
df1.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


### Printing the Shape of the DataFrame

In [13]:
df1.shape

(103, 3)

##### Geocoder Package not working

import geocoder
#import sys
#!{sys.executable} -m pip install geocoder

lat_lng_coords = None

df1['latitude'] = 2
df1['longitude'] = 3

for i in range(0,df1.shape[0]):
    # loop until you get the coordinates
    while(lat_lng_coords is None):
      print(i)    
      g = geocoder.google('{}'.format(df1['PostalCode'].iloc[i]))
      lat_lng_coords = g.latlng
    df1['latitude'].iloc[i] = lat_lng_coords[0]
    df1['longitude'].iloc[i] = lat_lng_coords[1]
df1.head()

#### Using a csv file to capture latitude and longitude

In [29]:
import os
os.chdir('C:\\Users\\ROHIT\\Desktop')
Data = pd.read_csv("Geospatial_Coordinates.csv")
Data = Data.rename(columns={'Postal Code':'PostalCode'})
Data.head()


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [38]:
df2 =pd.merge(df1,Data, on='PostalCode',how='inner')
df2.head()

,PostalCode,Borough,Neighborhood,latitude,longitude,Latitude,Longitude
0,M3A,North York,Parkwoods,2,3,43.753259,-79.329656
1,M4A,North York,Victoria Village,2,3,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",2,3,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",2,3,43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",2,3,43.662301,-79.389494


# Exploring Toronto Neighborhoods

In [39]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


#### Exploring Dataset

In [40]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df2['Borough'].unique()),
        df2.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


#### Creating an instance of Toronto Map

In [41]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [42]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto